In [2]:
import pandas as pd
import numpy as np
import ast  # For safely evaluating string representations of dictionaries
import json
import re

#### Data loading

In [113]:
df_meteo = pd.read_excel('data/meteo_history.xlsx')

In [114]:
# Function to convert strings to lists of dictionaries
def convert_to_list_of_dicts(item):
    try:
        return json.loads(item.replace("'", '"'))
    except (json.JSONDecodeError, AttributeError):
        return []

# Convert the strings in the 'region_phenomena' column to lists of dictionaries
df_meteo['region_phenomena'] = df_meteo['region_phenomena'].apply(convert_to_list_of_dicts)

In [115]:
df_meteo['article_date'] = pd.to_datetime(df_meteo['article_date'])

#### Group data by day - articles

In [116]:
grouped_by_day = df_meteo.groupby(df_meteo['article_date'].dt.date)

In [120]:
# Function to merge list of dictionaries for each region
def merge_dicts(dicts):
    result = {}
    for d in dicts:
        for key, value in d.items():
            if key not in result:
                result[key] = value
            else:
                # If the key exists, ensure the values are unique
                result[key] = list(set(result[key] + value))
    return [result]

In [121]:
df_agg = df_meteo.groupby('article_date').agg({'region_phenomena': lambda x: merge_dicts([item for sublist in x for item in sublist])}).reset_index()

In [124]:
# Extract date only (excluding time) from the article_date column
df_meteo['date_only'] = pd.to_datetime(df_meteo['article_date']).dt.date

In [125]:
def merge_dicts_in_list(dict_list):
    '''Merge dictionaries in a list'''
    result = {}
    for d in dict_list:
        for key, value in d.items():
            if key not in result:
                result[key] = value
            else:
                for subkey, sublist in value.items():
                    if subkey in result[key]:
                        result[key][subkey] = list(set(result[key][subkey] + sublist))
                    else:
                        result[key][subkey] = sublist
    return result

In [126]:
def merge_grouped_dicts(dict_list):
    combined_dict = {}
    # Iterate through each dictionary in the list
    for current_dict in dict_list:
        # Iterate through each key-value pair in the dictionary
        for key, value in current_dict.items():
            # If the key doesn't exist in the combined dictionary, just add it
            if key not in combined_dict:
                combined_dict[key] = value
            else:
                # Ensure that the 'value' is a dictionary before attempting to merge
                if isinstance(value, dict):
                    for subkey, sublist in value.items():
                        if subkey in combined_dict[key]:
                            # Merge lists without checking for uniqueness
                            combined_dict[key][subkey] = combined_dict[key][subkey] + sublist
                        else:
                            combined_dict[key][subkey] = sublist

    return combined_dict

In [127]:
# First merge dictionaries within each list in the 'region_phenomena' column
df_meteo['merged_phenomena'] = df_meteo['region_phenomena'].apply(merge_dicts_in_list)
# Now group by date and merge again
df_agg = df_meteo.groupby('date_only').agg({'merged_phenomena': merge_grouped_dicts}).reset_index()

In [128]:
# Extract just the date
df_meteo['date'] = pd.to_datetime(df_meteo['article_date']).dt.date

# Explode the 'region_phenomena' column
df_exploded = df_meteo.explode('region_phenomena')

In [131]:
def extract_key(d):
    if isinstance(d, dict) and d:
        return list(d.keys())[0]
    return None

def extract_value(d):
    if isinstance(d, dict) and d:
        return list(d.values())[0]
    return None

df_exploded['region'] = df_exploded['region_phenomena'].apply(extract_key)
df_exploded['phenomena'] = df_exploded['region_phenomena'].apply(extract_value)


In [132]:
# Filter out rows where 'region_phenomena' is NaN or an empty dictionary after explosion
df_exploded = df_exploded[df_exploded['region_phenomena'].notna() & df_exploded['region_phenomena'].astype(bool)]

# Transform the dictionary into separate columns
df_exploded['region'] = df_exploded['region_phenomena'].apply(lambda x: list(x.keys())[0])
df_exploded['phenomena'] = df_exploded['region_phenomena'].apply(lambda x: list(x.values())[0])

In [ ]:
# Drop unnecessary columns
df_exploded.drop(['article_date', 'region_phenomena'], axis=1, inplace=True)

# Group by date and region to aggregate phenomena
grouped = df_exploded.groupby(['date', 'region'])['phenomena'].sum().reset_index()

# Build the nested dictionary
result_dict = {}
for date, region, phenomena in grouped.values:
    if date not in result_dict:
        result_dict[date] = {}
    result_dict[date][region] = phenomena

print(result_dict)

In [134]:
# Convert dictionary to DataFrame
df_temp = pd.DataFrame(result_dict).T

# Reset index to have date as a column
df_temp = df_temp.reset_index()
df_temp.rename(columns={"index": "Date"}, inplace=True)

# Melt the DataFrame to have regions as a column
df_melted = df_temp.melt(id_vars="Date", 
                         var_name="Region", 
                         value_name="Phenomena")


In [135]:
df = pd.DataFrame(list(result_dict.items()), columns=['date', 'region_phenomena'])

In [137]:
df

,date,region_phenomena
0,2009-02-19,"{'Banat': ['Ninsori', 'Ninsoare', 'Ninsoare', ..."
1,2009-02-20,"{'Banat': [], 'Bucovina': ['Ninsoare', 'Vânt']..."
2,2009-02-21,"{'Banat': [], 'Bucovina': ['Ninsori', 'Ninsoar..."
3,2009-02-22,"{'Banat': ['Ninsoare', 'Caniculă'], 'Bucovina'..."
4,2009-02-23,"{'Banat': [], 'Bucovina': ['Ninsoare', 'Ninsor..."
...,...,...
4376,2023-08-15,"{'Banat': ['Averse'], 'Bucovina': ['Vreme inso..."
4377,2023-08-16,"{'Banat': ['Furtună', 'Vânt', 'Ploi', 'Grindin..."
4378,2023-08-17,"{'Banat': ['Averse', 'Grindină'], 'Bucovina': ..."
4379,2023-08-18,"{'Banat': ['Fulger', 'Trăsnet', 'Grindină'], '..."


In [136]:
df_params = pd.read_excel('data/date_judete.xlsx')

In [295]:
df.to_excel('data/regiuni_data.xlsx')

In [308]:
# Convert each dictionary in region_phenomena into a list of tuples
df['region_tuples'] = df['region_phenomena'].apply(lambda x: list(x.items()))

# Explode this list so each region has its own row
df = df.explode('region_tuples', ignore_index=True)

# Split each tuple into separate columns for region and phenomena
df[['region', 'phenomena']] = pd.DataFrame(df['region_tuples'].tolist(), index=df.index)

# Drop the original columns
df = df.drop(columns=['region_phenomena', 'region_tuples'])

# Filter out rows with empty phenomena lists
df = df[df['phenomena'].astype(bool)]

In [311]:
df_meteo_params = pd.read_excel('data/date_jud.xlsx')

In [335]:
df.reset_index(inplace=True)
df_meteo_params.reset_index(inplace=True)

In [336]:
df['date'] = pd.to_datetime(df['date'])
df_meteo_params['date'] = pd.to_datetime(df_meteo_params['date'])

In [354]:
df_concatenated = pd.merge(df, df_meteo_params, on=['date', 'region'])

In [365]:
df_concatenated = pd.read_excel('data/set_complet.xlsx')

#### Convert string to list and remove duplicates

In [367]:
def remove_duplicates_from_str_list(str_list):
    # Convert string representation of list to actual list
    actual_list = ast.literal_eval(str_list)
    # Return list without duplicates while maintaining order
    cleaned_list = []
    for item in actual_list:
        if item not in cleaned_list:
            cleaned_list.append(item)
    return cleaned_list

In [ ]:
df_concatenated['phenomena'] = df_concatenated['phenomena'].apply(remove_duplicates_from_str_list)

#### Function to remove duplicates from a list

In [375]:
def remove_duplicates_from_actual_list(lst):
    # Use dict.fromkeys() to preserve order and remove duplicates
    return list(dict.fromkeys(lst))


In [458]:
# Apply the function to the 'phenomena' column
df_concatenated_levels['phenomena'] = df_concatenated_levels['phenomena'].apply(remove_duplicates_from_actual_list)

In [462]:
df_concatenated_levels['phenomena'] = df_concatenated_levels['phenomena'].apply(lambda x: [item for item in x if item != 'Avalanșă'])

In [463]:
df_concatenated_levels.to_excel('data/set_complet_precip.xlsx')

#### Concatenate the Danube water levels with the data available data

In [391]:
df_danube = pd.read_excel('data/Cotele apelor Dunarii 6pct_prel.xlsx')

##### Compute the daily average of the water levels

In [403]:
df_danube['water_average'] = df_danube.iloc[:, 1:].mean(axis = 1)
df_danube.rename(columns = {'YMD': 'date'}, inplace=True)

In [411]:
df_danube['date'] = pd.to_datetime(df_danube['date'])

In [424]:
df_danube_copy = df_danube[['date', 'water_average']].copy()
df_concatenated_levels = pd.merge(df_concatenated, df_danube_copy, on = 'date', how = 'left')
df_concatenated_levels.drop('Unnamed: 0', axis = 1, inplace=True)

In [440]:
df_concatenated_levels.to_excel('data/set_complet_precip.xlsx')

#### Re-evaluate the dataset and maping phenomena

In [70]:
df_phen = pd.read_excel('data/set_complet_curatat.xlsx')

In [71]:
df_phen['phenomena'] = df_phen['phenomena'].apply(ast.literal_eval)

In [78]:
word_mapping = {
    'Ninsori': 'Ninsoare',
    'Ploi': 'Ploaie',
    'Vant puternic': 'Vant',
    'Vânt puternic': 'Vant',
    'Vant mai activ': 'Vant',
    'Ploi locale': 'Ploaie',
    'Descărcare electrică':'Fulger',
    'Descărcări electrice': 'Fulger',
    'Vijelii': 'Vijelie',
    'Furtuni': 'Furtuna',
    'Vant intens': 'Vant',
    'Descarcari electrice': 'Fulger',
    'Intensificări ale vântului': 'Vant',
    'Căderi de grindină': 'Grindina',
    'Grindină': 'Grindina',
    'Fenomene eletrice': 'Fulger', 
    'Ploi abundente': 'Ploaie',
    'Ploi consistente': 'Ploaie',
    'Ploi torentiale': 'Ploaie',
    'Frecvente descărcări electrice': 'Fulger',
    'Intensificări de scurtă durată ale vântului': 'Vant',
    'Vânt intensificat': 'Vant',
    'Ploi slabe': 'Ploaie',
    'Ploi de scurta durata': 'Ploaie',
    'Frecvente descarcari electrice': 'Fulger',
    'Intensificari ale vantului cu aspect de vijelie': 'Vijelie',
    'Caderi de grindina': 'Grindina',
    'Caniculă': 'Canicula',
    'Vânt intens': 'Vant',
    'Căldură insuportabilă': 'Canicula',
    'Fulgere' : 'Fulger',
    'Intensificari de scurta durata ale vantului cu aspect de vijelie': 'Vijelie',
    'Izolat grindină':'Grindina',
    'Vânt activ': 'Vant',
    'Averse locale': 'Averse',
    'Averse zdravene': 'Averse',
    'Vânt tare în zonele de munte': 'Vant',
    'Averse torențiale': 'Averse',
    'Strat consistent de zapada': 'Zapada',
    'Strat consistent de zăpadă': 'Zapada',
    'Ninsori': 'Ninsoare',
    'Ploi': 'Ploaie',
    'Ceața': 'Ceata',
    'Fulguială': 'Ninsoare',
    'Vânt': 'Vant',
    'Câțiva stropi': 'Ploaie',
    'Ninsori mixte': 'Ninsoare',
    'Ceată': 'Ceata',
    'Ceață': 'Ceata',
    'Intensificări de vânt': 'Vant',
    'Fenomene electrice': 'Fulger',
    'Fulgi de zăpadă': 'Ninsoare',
    'Fulguiei': 'Ninsoare',
    'Precipitații mixte': 'Lapovita',
    'Lapoviță': 'Lapovita',
    'Ploaie slabă': 'Ploaie',
    'Aversă': 'Averse',
    'Averse cu descărcări electrice': 'Averse',
    'Averse de ploaie': 'Averse',
    'Căderi de ploaie': 'Ploaie',
    'Rabufniri ale vântului': 'Vant',
    'Posibil chiar vijelii': 'Vijelie',
    'Porniri ale vantului': 'Vant',
    'Vânt cu aspect de vijelie': 'Vijelie',
    'Averse consistente': 'Averse',
    'Temporar va ploua': 'Ploaie',
    'Canitati importante de apa': 'Ploaie',
    'Cantitati mai insemnate de apa': 'Ploaie',
    'Canitati insemnate de apa': 'Ploaie',
    'Vant in rafale':'Vant',
    'Temperaturi ridicate':'Canicula',
    'Posibile ploi': 'Ploaie',
    'Ploi temporare': 'Ploaie',
    'Ploi mixte cu lapoviță și polei': 'Lapovita',
    'Ninsori mai linistite': 'Ninsoare',
    'Ninsori abia in noaptea de sambata spre duminica': 'Ninsoare',
    'Ninsori de-a lungul zilei': 'Ninsoare',
    'Ninsori locale': 'Ninsoare',
    'Ninsori troienite': 'Viscol',
    'Ninsori viscolite': 'Viscol',
    'Gero' : 'Ger',
    'Ninsoare mixtă':'Ninsoare',
    'Fulgi': 'Ninsoare',
    'Fulguieli': 'Ninsoare',
    'Ploi de vară': 'Ploaie',
    'Răbufniri ale vântului': 'Vant',
    'Porniri ale vântului': 'Vant',
    'Ploaie torențială': 'Ploaie',
    'Ploi torentiale': 'Ploaie',
    'Cantități mari de apă': 'Ploaie',
    'Precipitații slabe mixte': 'Lapovita',
    'Vânt slab': 'Vant',
    'Intensificări de scurtă durată': 'Vant',
    'Ploi insemnate cantitativ': 'Ploaie',
    'Ploi însemnate cantitaiv': 'Ploaie',
    'Averse cu fenomene electrice': 'Fulger',
    'Vreme foarte caldă': 'Canicula',
    'Atmosferă înăbușitoare': 'Canicula',
    'Ninsori abundente': 'Ninsori',
    'Mixt': 'Lapovita',
    'Precipitații slabe mixte': 'Lapovita',
    'ITU critic de 80': 'Canicula',
    'Posibile condiții de grindină': 'Grindina',
    'Grindină': 'Grindina',
    'Aversă torențială': 'Averse',
    'Precipitatii mixte': 'Lapovita',
    'Precipitatii salbe mixte': 'Lapovita',
    'Depuneri de polei': 'Polei',
    'Averse torentiale': 'Averse',
    'Ploi cu caracter de aversă': 'Ploaie',
    'Vant moderat':' Vant',
    'Vant nervos': 'Vant',
    'Ploi slabe cantitativ': 'Ploaie',
    'Ploi moderate cantitativ': 'Ploaie',
    'Posibil grindină': 'Grindina',
    'Rafale de vânt': 'Vant',
    'Intensificari ale vantului': 'Vant',
    'Vânt moderat': 'Vant',
    'Vânt scurt intensificat': 'Vant',
    'Temperaturi maxime record': 'Canicula',
    'Ploi zgomotoase': 'Ploaie',
    'Averse cu fulgere și vânt': 'Averse',
    'Intensificari de scurta durata ale vantului': 'Vant',
    'Izolat grindina':'Grindina',
    'Vant activ':'Vant',
    'Ploi rapide de vara':'Ploaie',
    'Ploi torențiale': 'Ploaie',
    'Ploi rapide de vară': 'Ploaie',
    'Ploi trecatoare': 'Ploaie',
    'Vânt moderat': 'Vant',
    'Cantități mai mari de apă': 'Ploaie',
    'Probabil grindină': 'Grindina',
    'Condiții de ploaie': 'Ploaie',
    'Fenomene de polei': 'Polei',
    'Precipitații sub formă de lapoviță și ninsoare': 'Lapovita',
    'Ploi pe suprafețe restrânse': 'Ploaie',
    'Picură din loc în loc': 'Ploaie',
    'Precipitații slabe, mixte': 'Lapovita',
    'Ploi izolate': 'Ploaie',
    'Ploi peste medie': 'Ploaie',
    'Averse slabe': 'Averse',
    'Vant cu rabufniri': 'Vant',
    'Ploi in zona de munte': 'Ploaie',
    'Ploi de vara':'Ploaie',
    'Averse isotite':'Averse',
    'Averse sporadice':'Averse',
    'Averse cu fulgere, vânt și grindină':'Averse',
    'Averse posibil torentiale':'Averse',
    'Ploi razlete':'Ploaie',
    'Ploi de scurtă durată':'Ploaie',
    'Ploi locale și temporare':'Ploaie',
    'Ploi serioase':'Ploaie',
    'Ploi sporadice':'Ploaie',
    'Averse locale spre seară':'Averse',
    'Formare de polei':'Polei',
    'Ninsori sporadice':'Ninsoare',
    'Ninsori slabe':'Ninsoare',
    'Ninsoare slabă':'Ninsoare',
    'Vant rece':'Vant',
    'Ninsoare slabă':'Ninsoare',
    'Ninsori slabe':'Ninsoare',
    'Fulguieli izolate':'Ninsoare',
    'Vant mai intens':'Vant',
    'Ninsoare slabă': 'Ninsoare',
    'Burniță/Fulguială': 'Burnita',
    'Burniță': 'Burnita',
    'Fulgi de zapada': 'Ninsoare',
    'Ninsori și lapoviță în seara și noaptea de vineri spre sâmbătă': 'Ninsoare',
    'Șanse reduse de ploaie': 'Ploaie',
    'Ploi în mai multe reprize': 'Ploaie',
    'Vant intensificat': 'Vant',
    'Valuri uriase': 'Valuri',
    'Descarcări electrice': 'Fulger',
    'Intensificări ale vântului cu aspect de vijelie': 'Vant',
    'Caldură insuportabilă': 'Canicula',
    'Disconfort termic foarte mare': 'Canicula',
    'Ploi trecătoare': 'Ploaie',
    'Disconfort termic ridicat': 'Canicula',
    'Caldură': 'Caldura',
    'Căldură': 'Caldura',
    'Temperaturi record': 'Canicula',
    'Averse cu descarcari electrice': 'Fulger',
    'Vant mai nervos': 'Vant',
    'Cod galben de caniculă':'Canicula',
    'Cod portocaliu de caniculă': 'Canicula',
    'Intensificări temporare ale vântului': 'Vant',
    'Ploi locale în Muntii Banatului': 'Ploaie',
    'Pericol de ploi moderate cantitativ': 'Ploaie',
    'Cod portocaliu: Vânt puternic și zăpadă troienită': 'Viscol',
    'Cod galben: Intensificări ale vântului și zăpadă spulberată':'Viscol',
    'Cod galben: Intensificări ale vântului și ninsori moderate cantitativ și viscolite':'Viscol',
    'Ploaie cu lapoviță': 'Lapovita',
    'Ninsori moderate cantitativ': 'Ninsoare',
    'Ceată densă': 'Ceata',
    'Cod galben de vânt intens': 'Vijelie',
    'Vânt mai insistent': 'Vant',
    'Izolat cu aspect de vijelie':'Vijelie',
    'Ploi de scurtă durată': 'Ploaie',
    'Rabufniri de vânt': 'Vant',
    'Rabufniri ale vantului': 'Vant',
    'Izolat de grindină': 'Grindina',
    'Cod portocaliu de inundatii': 'Inundatie',
    'Inundație': 'Inundatie',
    'Cod galben de inundatie': 'Inundatie',
    'Ploi mai zgomotoase': 'Ploaie',
    'Căldură toropitoare': 'Canicula',
    'Intensificari de vant': 'Vant',
    'Vânt insistent': 'Vant',
    'Ploi mixte cu polei': 'Polei',
    'Burnită': 'Burnita',
    'Burniță':'Burnita',
    'Burnița': 'Burnita',
    'Vant insistent': 'Vant',
    'Intensificări de scurtă durată ale vântului cu aspect de vijelie': 'Vijelie',
    'Intensificări puternice ale vântului cu aspect de vijelie': 'Vijelie',
    'Inundație': 'Inundatie',
    'Posibil vijelii': 'Vijelie',
    'Însorit': 'Vreme buna',
    'Vreme frumoasă': 'Vreme buna',
    'Vreme calda': 'Vreme buna',
    'Vreme caldă': 'Vreme buna',
    'Temperaturi în creștere': 'Vreme buna',
    'Averse cu fulgere': 'Fulger',
    'Stres termic': 'Canicula',
    'Racoare': 'Vreme rece',
    'Vânt nervos': 'Vant',
    'Răcire accentuată': 'Vreme rece',
    'Fulguieri': 'Ninsoare',
    'Înnorat': 'Innorat',
    'Temperaturi modeste': 'Vreme rece',
    'Vreme instabilă': 'Precipitatii',
    'Instabilitate atmosferică': 'Instabilitate atmosferica',
    'Viitură': 'Inundatie',
    'Averse cu tunete și fulgere': 'Fulger',
    'Averse trecatoare': 'Averse',
    'Mixtă': 'Lapovita',
    'Innorări persistente': 'Innorari',
    'Vreme insorită': 'Vreme buna',
    'Disconfort termic accentuat': 'Canicula',
    'Innorări trecătoare': 'Innorari',
    'Cer noros': 'Innorari',
    'Cer acoperit': 'Innorari', 
    'Vreme placuta': 'Vreme buna',
    'Frecvente descarcări electrice': 'Fulger',
    'Scădere temperaturi': 'Vreme rece',
    'Innorari temporare': 'Innorari',
    'Vreme plăcută': 'Vreme buna',
    'Izolat vijelii': 'Vijelie',
    'Instabilitate atmosferică temporar accentuată': 'Instabilitate atmosferica',
    'Vreme inchisa': 'Innorari',
    'Zăpadă spulberată': 'Viscol',
    'Temperaturi negative': 'Vreme rece',
    'Innorări temporare': 'Innorari',
    'Încălzire': 'Vreme buna',
    'Precipitații moderate': 'Precipitatii',
    'Precipitații excedentare': 'Precipitatii',
    'Vreme frumoasa':'Vreme buna',
    'Înnorări': 'Innorari',
    'Caldura': 'Canicula',
    'Ghețuș': 'Polei',
    'Vreme insorita':'Vreme buna',
    'Caderi de grindină': 'Grindina',
    'Vreme bună': 'Vreme buna',
    'Vreme insorita': 'Vreme buna',
    'Instabilitate atmosferică accentuată': 'Instabilitate atmosferica',
    'Incalzire': 'Vreme buna',
    'Racire': 'Vreme rece',
    'Zapada': 'Ninsoare',
    'Temperaturi scăzute': 'Vreme rece',
    'Vreme caldă': 'Vreme buna',
    'Zăpadă': 'Ninsoare',
    'Nori': 'Innorari',
    'Frig': 'Vreme rece',
    'Innorări': 'Innorari',
    'Disconfort termic': 'Canicula',
    'Precipitații slabe': 'Precipitatii',
    'Soare': 'Vreme buna',
    'Precipitații': 'Precipitatii',
    'Instabilitate atmosferică': 'Instabilitate atmosferica',
    'Vreme frumoasă': 'Vreme buna',
    'Innorat': 'Innorari',
    'Răcire': 'Vreme rece',
    'Instabilitate': 'Instabilitate atmosferica',
    'Negură': 'Ceata',
    'Gheață': 'Polei',
    'instabilitate atmosferica': 'Instabilitate atmosferica'
    
    # Add other mappings as required
}

#ramas ianuarie 2015

def process_word_list(phenomena_list):
    # Map words using the word_mapping dictionary
    processed_list = [word_mapping.get(word, word) for word in phenomena_list]
    # Remove duplicates while preserving order
    return list(dict.fromkeys(processed_list))



# Apply the function to the phenomena column
df_phen['phenomena'] = df_phen['phenomena'].apply(process_word_list)

### Get rid of the unnecessary phenomena

In [79]:
data_exploded = df_phen.copy()
data_exploded = data_exploded.explode('phenomena')

In [ ]:
# Counting occurrences for each phenomenon
phenomena_counts = data_exploded['phenomena'].value_counts()

# Identifying phenomena with less than 10 occurrences
rare_phenomena = phenomena_counts[phenomena_counts < 10].index

# Filtering out the rare phenomena from the exploded dataframe
data_filtered = data_exploded[~data_exploded['phenomena'].isin(rare_phenomena)]

data_filtered['phenomena'].value_counts()

In [ ]:
# List of phenomena to be removed
remove_phenomena = ['Uragan', 'Alunecare', 'Cod galben', 'Cod portocaliu', 'Eroziune', 'Cutremur']

# Filtering the exploded dataset
filtered_data_remove_outliers = data_filtered[~data_filtered['phenomena'].isin(remove_phenomena)]

# Checking the updated dataset
filtered_data_remove_outliers['phenomena'].value_counts()

In [ ]:
df_phen = filtered_data_remove_outliers.groupby(['date', 'region', 'county', 'pm10_quality', 'air_pressure', 'temperature',
                                      'humidity']
                                     ).agg({'phenomena': list}).reset_index()

### Translate every phenomenon in English

In [84]:
weather_translation = {
    'Ninsoare': 'Snow',
    'Vant': 'Wind',
    'Ploaie': 'Rain',
    'Lapovita': 'Sleet',
    'Inundatie': 'Flood',
    'Ceata': 'Fog',
    'Ger': 'Frost',
    'Viscol': 'Blizzard',
    'Polei': 'Freezing rain',
    'Fulger': 'Lightning',
    'Innorari': 'Cloudiness',
    'Vreme rece': 'Cold weather',
    'Burnita': 'Drizzle',
    'Cer variabil': 'Variable sky',
    'Cer senin': 'Clear sky',
    'Vreme buna': 'Good weather',
    'Precipitatii': 'Precipitation',
    'Vijelie': 'Whirlwind',
    'Canicula': 'Heatwave',
    'Averse': 'Showers',
    'Furtună': 'Storm',
    'Grindina': 'Hail',
    'Tunete': 'Thunder',
    'Secetă': 'Drought',
    'Vreme schimbătoare': 'Changeable weather',
    'Instabilitate atmosferica': 'Atmospheric instability',
    'Ciclon': 'Cyclone',
    'Chiciură': 'Rime',
    'Trăsnet': 'Thunderbolt',
    'Tunet': 'Thunder',
    'Brumă': 'Frost',
    'Incendii': 'Fires',
    'Ploaie înghețată': 'Frozen rain',
    'Tornadă': 'Tornado'
}

def translate_word_list(phenomena_list):
    # Map words using the weather_translation dictionary
    processed_list = [weather_translation.get(word, word) for word in phenomena_list]
    # Remove duplicates while preserving order
    return list(dict.fromkeys(processed_list))

# Apply the function to the phenomena column
df_phen['phenomena'] = df_phen['phenomena'].apply(translate_word_list)

In [86]:
df_phen.to_excel('data/set_complet_curatat_tradus.xlsx', index=False)

In [145]:
coords = pd.read_excel('data/coordonate.xlsx')

In [146]:
coords

,county,lat,long
0,TM,45.66667,21.50000
1,CT,44.25000,28.33333
2,IS,47.25000,27.25000
3,B,44.50000,26.08333
4,DJ,44.16667,23.75000
5,CJ,46.64944,23.81836
6,BV,45.75000,25.33333


In [149]:
df_merged = pd.merge(df_phen, coords, on="county", how="left")

# Display the first few rows of the merged dataset to verify the merge
df_merged.head()

,date,region,phenomena,county,42i - NO2_Valoare [µg/m³],43i - SO2_Valoare [µg/m³],48i - CO_Valoare [mg/m³],GRAV. 10 - PM 10_Valoare [µg/m³],GRAV. 2.5 - PM 2.5_Valoare [µg/m³],MTX - Presiunea aerului_Valoare [mbar],MTX - Temperatura aer_Valoare [°C],MTX - Umiditate relativa_Valoare [%],water_average,lat,long
0,2009-02-19,Banat,"[Ninsoare, Vant, Alunecare]",TM,NaN,NaN,NaN,51.630000,19.98,995.400,-4.095,85.50,NaN,45.66667,21.50000
1,2009-02-19,Dobrogea,"[Ploaie, Lapovita, Vant, Ninsoare, Inundatie, ...",CT,NaN,NaN,NaN,21.520000,NaN,1005.875,6.745,96.75,NaN,44.25000,28.33333
2,2009-02-19,Moldova,"[Ninsoare, Ger, Ploaie, Lapovita, Vant, Inunda...",IS,NaN,NaN,NaN,44.600000,30.46,1010.700,-0.935,99.00,NaN,47.25000,27.25000
3,2009-02-19,Muntenia,"[Ninsoare, Lapovita, Ploaie, Vant, Inundatie, ...",B,29.33375,13.971667,0.673333,34.428571,NaN,998.880,0.252,86.20,NaN,44.50000,26.08333
4,2009-02-19,Oltenia,"[Ploaie, Lapovita, Ninsoare, Vant]",DJ,NaN,NaN,NaN,13.265000,22.18,NaN,0.760,97.00,NaN,44.16667,23.75000


In [150]:
df_merged.to_excel('data/set_corectat_complet.xlsx')

In [152]:
df_merged['date'] = pd.to_datetime(df_merged['date'])

In [155]:
df_merged['date'].dt.year.value_counts()

date
2021    2309
2018    2264
2019    2240
2016    2209
2020    2195
2017    2176
2022    2093
2015    1912
2009    1597
2010    1583
2014    1555
2023    1457
2011    1002
2012     708
2013     561
Name: count, dtype: int64